In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import seaborn as sns
import csv
import monai
from monai.transforms import \
    Compose, LoadImaged, AddChanneld, Orientationd, \
    Spacingd, \
    ToTensord,  \
    DataStatsd, \
    ToDeviced
from monai.data import list_data_collate
import torch
import pytorch_lightning as pl
from torchsummary import summary
monai.config.print_config()
import sys
# sys.path.append(r'/data16/private/zc348/project/DL_HMC_attention/mCT/util/python')
sys.path.append(r'/data16/private/zc348/project/DL_HMC_attention/util/python')
import vicra_toolbox
import nibabel; nibabel.imageglobals.logger.setLevel(40)
# New transforms 
import torch.nn as nn
import torch.nn.functional as F
sys.path.append(r'../')

torch.set_num_threads(4)
from dlhmc.transforms import (
    CreateImageStack,
    ComputeRelativeMotion,
    RandSamplePET,
    ComputeRelativeMotiond,
    CreateImageStackd,
    RandSamplePETd,
)

from dlhmc.utils.data import (
    concatenate_vicra,
    split_dataset
)



2025-05-06 22:11:02,370 - Created a temporary directory at /tmp/tmpoxzglzef
2025-05-06 22:11:02,375 - Writing /tmp/tmpoxzglzef/_remote_module_non_scriptable.py
MONAI version: 1.0.1
Numpy version: 1.23.4
Pytorch version: 1.13.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 8271a193229fe4437026185e218d5b06f7c8ce69
MONAI __file__: /home1/zc348/anaconda3/envs/dl-hmc_2301c/lib/python3.9/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 4.0.2
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 9.2.0
Tensorboard version: 2.11.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.14.1
tqdm version: 4.64.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.0
pandas version: 1.4.3
einops version: 0.6.0
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd versi

In [ ]:
tr_set_all=[]# all case name

test_set=[]# testing case name

tr_set = tr_set_all[0:100]
from dataset_summary_toolbox import compute_delta_T 
from data_prep_toolbox import delta_T_magnitude, Relative_motion_A_to_B_12, build_legal_dataset, deal_dataframe, clean_df
from sampling_toolbox import data_split_sample, add_T_deltaT

# Get the summaries

summaries, delta_T_all=compute_delta_T(['APP311'], tr_set, data_type = 'real')

def calculate_beta(df):
    nums = df['T'].to_numpy()
    trans, rot = 0, 0
    for array in range(len(nums)):
        for i in range(3):
            trans += abs(nums[array][i])
        for j in range(3):
            rot += abs(nums[array][3 + j])
#     print("trans:", trans, "rot:", rot)
    beta = trans/(trans + rot)
#     print('beta:', beta)
    return beta

num_subsample = 360 #1800 # Number of samples per subject (to be split between training, validation and testing sets)

train_list=list()
val_list=list()
test_list=list()

# Sample training, validation and testing sets and save them in lists
for summary in summaries: 
    summary['VICRA']=concatenate_vicra(summary)

    print('Dataset contains total {:,d} entries'.format(len(summary)))
    df_train, df_val, df_test = split_dataset(summary, num_subsample=num_subsample, test_size_percent=0.1, validation_size_percent=0.1)
    
    
    print('Dataset split into 3 subsets:')
    print(' Train set size: {:,d}'.format(len(df_train)))
    print(' Val set size: {:,d}'.format(len(df_val)))
    print(' Test set size: {:,d}'.format(len(df_test)))
    
    train_dict={
        'DataFrame':df_train
    }

    val_dict={
        'DataFrame':df_val
    }
    
    test_dict={
        'DataFrame':df_test
    }
    
    train_list.append(train_dict)
    val_list.append(val_dict)
    test_list.append(test_dict)


    
# SIZE = (64,64,64)
SIZE = (32,32,32)

image_key = "ThreeD_Cloud"
meta_keys=['ScanStart','VICRA']

# Need to format as a dictionary for input
# sample_dict = {
#     'DataFrame': df_train,
    # 'PatientID': df_sample['PatientID'].unique()
# }

# Here, we only have one subject of data, so a small example, but more subjects can be added
# to this list.
# sample_data_dict = [sample_dict]

pet_transforms = monai.transforms.Compose([
    CreateImageStackd(keys="DataFrame", image_key=image_key, spatial_size=SIZE),
    RandSamplePETd(keys=image_key, meta_data_key="DataFrame", meta_keys=meta_keys, num_samples=8), 
    ComputeRelativeMotiond(keys=['VICRA_ref','VICRA_mov'], output_key='VICRA_rel'),
    monai.transforms.DeleteItemsd(keys=["DataFrame", "ScanStart", "VICRA_ref", "VICRA_mov", image_key]),
    monai.transforms.ToTensord(
        keys=["ScanStart_ref", "ScanStart_mov", "VICRA_rel"]
    )
])



train_ds = monai.data.PersistentDataset(
    data=train_list, 
    transform=pet_transforms, 
    cache_dir="/data16/private/zc348/project/dev/100hrrt/cache/"
)

train_loader = torch.utils.data.DataLoader(
    train_ds, 
    batch_size=12, 
    shuffle=True, 
    num_workers=4, 
    collate_fn=monai.data.list_data_collate
)

val_ds = monai.data.PersistentDataset(
    data=val_list, 
    transform=pet_transforms, 
    cache_dir="/data16/private/zc348/project/dev/100hrrt/cache/"
)

val_loader = torch.utils.data.DataLoader(
    val_ds, 
    batch_size=12, 
    shuffle=True, 
    num_workers=4, 
    collate_fn=monai.data.list_data_collate
)







In [6]:
from dlhmc.utils import Relative_motion_A_to_B_12,RotTransMatrix_6Params

def matrix_transformation(gt_reg):
    i_6_list = list()
    for i_12 in gt_reg:
        i_6 = np.reshape(i_12,12)
        i_6 = torch.from_numpy(RotTransMatrix_6Params(i_6,1))
        i_6_list.append(i_6)
        
    gt_reg_6 = torch.stack(i_6_list).float()
    # return gt_reg_6
    return gt_reg_6.to(gt_reg.get_device())

In [ ]:
class UNet(nn.Module):

    def __init__(self, in_channel, out_channel):
        super(UNet, self).__init__()
        #Encode
        self.conv_encode1 = self.contracting_block(in_channels=in_channel, out_channels=32)
        self.conv_maxpool1 = torch.nn.MaxPool3d(kernel_size=2)
        self.conv_encode2 = self.contracting_block(32, 64)
        self.conv_maxpool2 = torch.nn.MaxPool3d(kernel_size=2)
        self.conv_encode3 = self.contracting_block(64, 128)
        self.conv_maxpool3 = torch.nn.MaxPool3d(kernel_size=2)


    def contracting_block(self, in_channels, out_channels, kernel_size=3):
        """
        This function creates one contracting block
        """
        block = torch.nn.Sequential(
            torch.nn.Conv3d(kernel_size=kernel_size, in_channels=in_channels, out_channels=out_channels, padding=1),
            torch.nn.BatchNorm3d(out_channels),
            torch.nn.ReLU(),
            torch.nn.Conv3d(kernel_size=kernel_size, in_channels=out_channels, out_channels=out_channels, padding=1),
            torch.nn.BatchNorm3d(out_channels),
            torch.nn.ReLU(),
        )
        return block
    
    def forward(self, x):
        # Encode
        encode_block1 = self.conv_encode1(x)
        encode_pool1 = self.conv_maxpool1(encode_block1)
        encode_block2 = self.conv_encode2(encode_pool1)
        encode_pool2 = self.conv_maxpool2(encode_block2)
        
        encode_block3 = self.conv_encode3(encode_pool2)
        encode_pool3 = self.conv_maxpool3(encode_block3)


        return encode_pool3

In [ ]:
class Cross_attention(nn.Module):
    def  __init__(self, all_channel=128, all_dim=4*4*4):	

        super(Cross_attention, self).__init__()
        self.linear_e = nn.Linear(all_channel, all_channel,bias = False)
        self.channel = all_channel
        self.dim = all_dim
        self.gate = nn.Conv3d(all_channel, 1, kernel_size  = 1, bias = False)
        self.gate_s = nn.Sigmoid()
        self.conv1 = nn.Conv3d(all_channel*1, all_channel, kernel_size=3, padding=1, bias = False)
        self.conv2 = nn.Conv3d(all_channel*1, all_channel, kernel_size=3, padding=1, bias = False)
        self.bn1 = nn.BatchNorm3d(all_channel)
        self.bn2 = nn.BatchNorm3d(all_channel)
        self.prelu = nn.ReLU(inplace=True)
        self.conv3d_7 = nn.Conv3d(in_channels=all_channel *2 , out_channels=all_channel, kernel_size=1, stride=(1, 1, 1), padding=0)
        self.pathC_bn1 = nn.BatchNorm3d(all_channel*2)
        self.conv3d_8 = nn.Conv3d(in_channels=all_channel, out_channels=all_channel//2, kernel_size=3, stride=(1, 1, 1), padding=1)
        self.conv3d_9 = nn.Conv3d(in_channels=all_channel//2, out_channels=all_channel//8, kernel_size=3, stride=(1, 1, 1), padding=1)
        self.pathC_bn2 = nn.BatchNorm3d(all_channel//8)

        self.conva = nn.Conv3d(all_channel, all_channel, kernel_size=1, padding=0, bias = False)
        self.convb = nn.Conv3d(all_channel, all_channel, kernel_size=1, padding=0, bias = False)
    
    
		
    def forward(self, exemplar, query): 
        
	 
        fea_size = query.size()[2:]	 
        
        #### correlation matrix computation
        exemplar_v =  self.convb(exemplar) 
        query_v =  self.convb(query) 
        exemplar_q =  self.conva(exemplar) 
        query_k =  self.conva(query) 
        exemplar_flat = exemplar_q.view(-1, self.channel, self.dim) #N,C,H*W
        query_flat = query_k.view(-1, self.channel, self.dim)
        exemplar_t = torch.transpose(exemplar_flat,1,2).contiguous()  #batch size x dim x num
        A = torch.bmm(exemplar_t, query_flat)
        A = F.softmax(A, dim = 1) 
        B = F.softmax(torch.transpose(A,1,2),dim=1)
        query_att = torch.bmm(exemplar_v.view(-1, self.channel, self.dim) , A).contiguous() 
        exemplar_att = torch.bmm(query_v.view(-1, self.channel, self.dim), B).contiguous()
        #####self-gate mechanism
        input1_att = exemplar_att.view(-1, self.channel, fea_size[0], fea_size[1], fea_size[2])  
        input2_att = query_att.view(-1, self.channel, fea_size[0], fea_size[1], fea_size[2])
        input1_mask = self.gate(input1_att)
        input2_mask = self.gate(input2_att)
        input1_mask = self.gate_s(input1_mask)
        input2_mask = self.gate_s(input2_mask)
        input1_att = input1_att * input1_mask
        input2_att = input2_att * input2_mask
        input1_att = input1_att +  exemplar_v
        input2_att = input2_att + query_v
        ######Deep norm and fusion
        input1_att  = self.conv1(input1_att )
        input2_att  = self.conv2(input2_att ) 
        input1_att  = self.bn1(input1_att )
        input2_att  = self.bn2(input2_att )
        input1_att  = self.prelu(input1_att )
        input2_att  = self.prelu(input2_att )

        conv_input = torch.cat((input1_att, input2_att), 1)
        x = self.pathC_bn1(conv_input)
        x = self.conv3d_7(x)
        x = self.prelu(x)
        x = self.conv3d_8(x)
        x = self.prelu(x)
        x = self.conv3d_9(x)
        x = self.pathC_bn2(x)

        return x.view(x.size()[0],-1)

In [ ]:
class cross_att_dataloader(pl.LightningModule):
    
    def __init__(self, dropout=0.3,img_size=32):
        super().__init__()
        
             
        self.dropout = dropout
        
        self.feature_extractor = UNet(1,1)

        self.coattention = Cross_attention()


        self.regression_layers = torch.nn.Sequential(

            torch.nn.Linear(1024, 128),
            torch.nn.Linear(128, 16),
            torch.nn.Linear(16, 6),
        )
        self.loss_function = torch.nn.MSELoss()
        
    def forward(self, x1, x2):

        y1 = self.feature_extractor(x1)
        y2 = self.feature_extractor(x2)
        y = self.coattention(y1,y2)
        y = self.regression_layers(y)
        return y
    
    
    
    def prepare_data(self):
        # set deterministic training for reproducibility
        monai.utils.misc.set_determinism(seed=42)
    
    def training_step(self, batch, batch_idx):
        x1 = batch["ThreeD_Cloud_ref"]
        x2 = batch["ThreeD_Cloud_mov"]
        ref_time = batch['ScanStart_ref']
        mov_time = batch['ScanStart_mov']
        # x_t = torch.stack([ref_time, mov_time], dim=1)
        gt_reg = batch["VICRA_rel"].float()
        y = self.forward(x1,x2)
       
        target_six = matrix_transformation(gt_reg)
        loss = self.loss_function(y, target_six)   

        return {"loss": loss}

        
    def training_epoch_end(self, outputs):
        # Calculate the average loss
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        # Logging at the end of every epoch
        self.logger.experiment.add_scalar('Train/Loss', avg_loss, self.current_epoch)
    
    def validation_step(self, batch, batch_idx):
        x1 = batch["ThreeD_Cloud_ref"]
        x2 = batch["ThreeD_Cloud_mov"]
        ref_time = batch['ScanStart_ref']
        mov_time = batch['ScanStart_mov']
        x_t = torch.stack([ref_time, mov_time], dim=1)
        gt_reg = batch["VICRA_rel"].float()
        y = self.forward(x1,x2)
        target_six = matrix_transformation(gt_reg)
        loss = self.loss_function(y, target_six)   

        return {"val_loss": loss}

    def validation_epoch_end(self, outputs):
        # Calculate the average loss
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        # Logging at the end of every epoch
        self.logger.experiment.add_scalar('Val/Loss', avg_loss, self.current_epoch)

        # Log the value for model checkpoint saving
        self.log('val_loss', avg_loss.item()) #added .item(), otherwise validation wouldn't work 

    def configure_optimizers(self):
        total_params = list(self.feature_extractor.parameters()) + list(self.regression_layers.parameters()) + list(self.coattention.parameters())
        opt = torch.optim.Adam(total_params, lr=5e-4)
        scheduler = {'scheduler': torch.optim.lr_scheduler.StepLR(optimizer=opt, step_size=200, gamma=0.98),
                     'name': 'Learning Rate'}
        return [opt], [scheduler]

In [ ]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

model = cross_att_dataloader().to(device)


from torchinfo import summary



MODEL_PATH = os.path.join('.','saved_model')
print('MODEL_PATH={}'.format(MODEL_PATH))


# Set up loggers and checkpoints
tb_logger = pl.loggers.TensorBoardLogger(
    save_dir=os.path.join(MODEL_PATH,'logs')
)

lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='epoch')

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=os.path.join(MODEL_PATH),
    filename="PETRegNet-{epoch}-{val_loss:.3f}",
    monitor='val_loss',
    save_last=True,
    save_top_k=2,
    mode='min'
)

# Check for last checkpoint
resume_checkpoint = None
if os.path.exists(os.path.join(MODEL_PATH,'last.ckpt')):
    resume_checkpoint = os.path.join(MODEL_PATH,'last.ckpt')

# Initialise Lightning's trainer.
trainer = pl.Trainer(
    accelerator='gpu',
    devices=[1],
    max_epochs=3000, #10k is generally enough for single-subjct studies
    logger=tb_logger,
    callbacks=[lr_monitor,checkpoint_callback],
    num_sanity_val_steps=1,
    check_val_every_n_epoch=20,
#     resume_from_checkpoint=resume_checkpoint # deprecated since v1.5
)

# Train
trainer.fit(model, train_loader, val_loader, ckpt_path=resume_checkpoint) # previously was trainer.fit(model, train_dataloader=tr_loader, val_dataloaders=val_loader)